In [1]:
# Importamos los paquetes necesarios
import pandas as pd
import numpy as np





In [2]:
# Cargamos la base de datos que produjimos en el tp anterior
df = pd.read_excel("eph_tp2.xlsx")
# Volvemos a calcular los datos de pobreza con todos los individuos ya que antes excluimos las observaciones que poseían ITF = 0
# Creamos la columna que totaliza los adultos equivalentes que componen cada hogar
df['ad_equiv_hogar'] = df.groupby("CODUSU")["adulto_equiv"].transform("sum")
# Añadimos una columna con los requerimientos monetarios de cada hogar
cbt2005 = 205.07 # Canasta básica total
cbt2025 = 365177
df['ingreso_necesario'] = 0 # Creamos la columna con 0
# Calculamos el ingreso necesario por hogar para 2005
df.loc[df['ANO4'] == 2005, 'ingreso_necesario'] = (cbt2005 * df.loc[df['ANO4'] == 2005, 'ad_equiv_hogar'])
# Calculamos el ingreso necesario por hogar para 2025
df.loc[df['ANO4'] == 2025, 'ingreso_necesario'] = (cbt2025 * df.loc[df['ANO4'] == 2025, 'ad_equiv_hogar'])
# Creamos una columna que indique 1 si el individuo pertecene a un hogar pobre y 0 en caso contrario
df['pobre'] = (df['ITF'] < df['ingreso_necesario']).astype(int) # Primero nos devuelve true o false y lo convertimos a 1-0
# Reemplazamos la clasificación de pobre por nan para aquellas observaciones que tienen un ITF = 0
df.loc[df['ITF'] == 0, 'pobre'] = np.nan

# Creamos la variable de edad al cuadrado
df['edad_cuadrado'] = df['CH06']*df['CH06']

# Calculamos los años de educación
jardin = 2
primaria = 8 # Primaria Bsas (6+2)
egb = 11 # Jardin + 9 años egb (9+2)
secundaria = 14 # Jardin + primaria + secundaria 6 años
polimodal = 14 # Jardin + egb + polimodal 3 años
terciario = 17 # 14 de secundaria o polimodal + 3
universitario = 19 # 14 de secundaria o polimodal + 5
posgrado = 22 # 19 de universitario + 3 años
especial = 14



def calcular_educ(row):
    if row['CH12'] == 0 or row['CH12'] == 99: # No hay dato
        return np.nan
    elif row['CH13'] == 0 or row['CH13'] == 9 or row['CH13'] == 1: # Termino el nivel o no hay dato y asumimos que termino el nivel anterior
        if row['CH12'] == 1:
            return jardin
        elif row['CH12'] == 2:
            return primaria
        elif row['CH12'] == 3:
            return egb
        elif row['CH12'] == 4:
            return secundaria
        elif row['CH12'] == 5:
            return polimodal
        elif row['CH12'] == 6:
            return terciario
        elif row['CH12'] == 7:
            return universitario
        elif row['CH12'] == 8:
            return posgrado
        elif row['CH12'] == 9:
            return especial
    elif row['CH13'] == 2: # No Termino el nivel
        if ['CH14'] == 99 or row['CH14'] == 98:
            if row['CH12'] == 1:
                return jardin - 1
            elif row['CH12'] == 2:
                return primaria - 1
            elif row['CH12'] == 3:
                return egb - 1
            elif row['CH12'] == 4:
                return primaria - 1
            elif row['CH12'] == 5:
                return primaria - 1
            elif row['CH12'] == 6:
                return terciario - 1
            elif row['CH12'] == 7:
                return universitario - 1
            elif row['CH12'] == 8:
                return posgrado - 1
            elif row['CH12'] == 9:
                return especial - 1
        elif row['CH14'] == 0 or row['CH14'] == 1 or row['CH14'] == 2 or row['CH14'] == 3 or row['CH14'] == 4 or row['CH14'] == 5 or row['CH14'] == 6 or row['CH14'] == 7 or row['CH14'] == 8 or row['CH14'] == 9:
                if row['CH12'] == 1:
                    return row['CH14']
                elif row['CH12'] == 2:
                    return jardin + row['CH14']
                elif row['CH12'] == 3:
                    return jardin + row['CH14']
                elif row['CH12'] == 4:
                    return secundaria + row['CH14']
                elif row['CH12'] == 5:
                    return polimodal + row['CH14']
                elif row['CH12'] == 6:
                    return secundaria + row['CH14']
                elif row['CH12'] == 7:
                    return secundaria + row['CH14']
                elif row['CH12'] == 8:
                    return universitario + row['CH14']
                elif row['CH12'] == 9:
                    return row['CH14']

df['educ'] = df.apply(calcular_educ, axis=1)




/tmp/ipython-input-3143834477.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[151.7518 436.7991 436.7991 ... 674.6803 674.6803 674.6803]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['ANO4'] == 2005, 'ingreso_necesario'] = (cbt2005 * df.loc[df['ANO4'] == 2005, 'ad_equiv_hogar'])


In [3]:
df

,CODUSU,NRO_HOGAR,COMPONENTE,REGION,ANO4,CH04,CH06,CH07,CH08,CH12,...,P21,P47T,ITF,IPCF,adulto_equiv,ad_equiv_hogar,ingreso_necesario,pobre,edad_cuadrado,educ
0,125098,1,1,1,2005,1,80,4,1,4,...,0,583,583,583,0.74,0.74,1.517518e+02,0.0,6400,16.0
1,125188,1,1,1,2005,1,26,2,4,4,...,300,450,450,150,1.02,2.13,4.367991e+02,0.0,676,14.0
2,125188,1,2,1,2005,2,27,2,4,6,...,0,0,450,150,0.76,2.13,4.367991e+02,0.0,729,17.0
3,125188,1,3,1,2005,2,0,5,4,99,...,0,0,450,150,0.35,2.13,4.367991e+02,0.0,0,NaN
4,125242,1,1,1,2005,2,71,4,1,2,...,0,855,855,855,0.67,0.67,1.373969e+02,0.0,5041,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16660,TQRMNORWYHKMPRCDEIJAH00860437,1,1,1,2025,1,49,2,1,4,...,700000,700000,0,0,1.00,4.18,1.526440e+06,NaN,2401,NaN
16661,TQRMNORWYHKMPRCDEIJAH00860437,1,2,1,2025,2,51,2,1,6,...,500000,500000,0,0,0.76,4.18,1.526440e+06,NaN,2601,17.0
16662,TQRMNORWYHKMPRCDEIJAH00860437,1,3,1,2025,2,22,5,1,6,...,-9,-9,0,0,0.76,4.18,1.526440e+06,NaN,484,17.0
16663,TQRMNORWYHKMPRCDEIJAH00860437,1,4,1,2025,1,26,5,1,4,...,0,0,0,0,1.02,4.18,1.526440e+06,NaN,676,19.0


In [4]:
#presentamos una estadistica descriptiva para la variable educ
descriptive_stats = df['educ'].describe()
display(descriptive_stats)

,educ
count,15692.000000
mean,12.281481
std,5.240672
min,0.000000
25%,8.000000
50%,14.000000
75%,17.000000
max,24.000000


**Comentarios sobre la variable 'educ'

La variable educ presenta una tendencia central en torno a la educación secundaria completa, con un alto nivel de dispersión y la existencia de personas que no han cursado educación formal así como otras que tienen estudios universitarios o de posgrado

## ejercicio 5 tabla de resumen


In [6]:
#calculamos el total de observaciones y el numero de observaciones para cada año
total_observations = len(df_yyy)
total_obs_2005 = len(df_yyy[df_yyy['ANO4'] == 2005])
total_obs_2025 = len(df_yyy[df_yyy['ANO4'] == 2025])

In [7]:
#calculamos el numero de valores fatantes en la columna pobre para todos los años
nan_pobre_total = df_yyy['pobre'].isnull().sum()
nan_pobre_2005 = df_yyy[df_yyy['ANO4'] == 2005]['pobre'].isnull().sum()
nan_pobre_2025 = df_yyy[df_yyy['ANO4'] == 2025]['pobre'].isnull().sum()

In [19]:
#calculamos pobres y no pobres
#'pobre' es 1 (pobre) y 'pobre' es 0 (no pobre)
poor_total = df_yyy['pobre'].sum()
non_poor_total = (df_yyy['pobre'] == 0).sum()
poor_2005 = df_yyy[df_yyy['ANO4'] == 2005]['pobre'].sum()
non_poor_2005 = (df_yyy[df_yyy['ANO4'] == 2005]['pobre'] == 0).sum()
poor_2025 = df_yyy[df_yyy['ANO4'] == 2025]['pobre'].sum()
non_poor_2025 = (df_yyy[df_yyy['ANO4'] == 2025]['pobre'] == 0).sum()

In [20]:
#contamos las variables limpias y homogeneizadas
cleaned_variables_count = 5

In [21]:
#creamos una tabla de resumen
summary_data = {
    2005: [int(total_obs_2005), int(nan_pobre_2005), int(poor_2005), int(non_poor_2005), int(cleaned_variables_count)],
    2025: [int(total_obs_2025), int(nan_pobre_2025), int(poor_2025), int(non_poor_2025), int(cleaned_variables_count)],
    'Total': [int(total_observations), int(nan_pobre_total), int(poor_total), int(non_poor_total), int(cleaned_variables_count)]
}
summary_table = pd.DataFrame(summary_data, index=["Cantidad observaciones", "Cantidad de observaciones con NAs en la variable “Pobre”", "Cantidad de Pobres", "Cantidad de No Pobres", "Cantidad de variables limpias y homogeneizadas"])
display(summary_table)

,2005,2025,Total
Cantidad observaciones,9484,7181,16665
Cantidad de observaciones con NAs en la variable “Pobre”,113,2872,2985
Cantidad de Pobres,2532,1341,3873
Cantidad de No Pobres,6839,2968,9807
Cantidad de variables limpias y homogeneizadas,5,5,5
